# Options Screener

This notebook is meant to analyze a list of tickers and output which tickers have the highest value call options with the highest probability of increasing in value in the short-term.  

Manual steps procedure:  
- Update options_list2.csv with the latest OI pct changes from Optionistics.com using the options_list_compare.xlsx file  
- Get the latest finviz file from the local directory  

This screener looks at the following attributes:  
- Whether an option is 'cheap' i.e. has low realized volatility compared to its historical realized volatility, the lowest 15% are given a point  
- Implied volatility premium or discount, the top 15% are given a point  
- Open interest % change from optionistics, these are given a point  
- Highest float short percent, the top 15% are given a point

### REMEMBER TO UPDATE THE OPTIONS_LIST2.CSV MANUALLY, BE SURE TO LOG IN FIRST.

In [1]:
# %matplotlib inline

import math
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
import time

tic = time.perf_counter()

In [3]:
import pandas as pd

tickers_df = pd.read_csv('../0_inputs/asset_screeners/options_list1.csv')
optionistics_data = pd.read_csv('../0_inputs/asset_screeners/options_list2.csv')
# optionistics_data.head(3)

ticker_dictionary = {
    'VIX':'^VIX'
}

ticker_list_df = pd.merge(tickers_df[['ticker']],optionistics_data[['ticker']],on='ticker',how='outer')

ticker_list = ticker_list_df['ticker'].to_list()

In [4]:
def realized_vol(price_data, window=30):

#     print(price_data.tail(5))
    log_return = (price_data["Close"] / price_data["Close"].shift(1)).apply(np.log)

    return log_return.rolling(window=window, center=False).std() * math.sqrt(252)

In [5]:
# windows = [30, 60, 90, 120] # calendar days
windows = [22, 44, 66, 88] # trading days
quantiles = [0.25, 0.75]

min_ = []
max_ = []
median = []
top_q = []
bottom_q = []
realized = []

In [6]:
completed_tkr_list = []
realized_vol_list = []
avg_pct_list = []
pct30_list = []
pct60_list = []
pct90_list = []
pct120_list = []
counter = 1

cur_price_dict = {}

# ticker_list = ['AAPL','TSLA','MSFT'] # test list

for ticker in ticker_list:

    print(f"Processing {ticker}, {counter} of {len(ticker_list)}...")
    
    try:
        # data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

        # start="2020-01-01", end="2020-12-31"

        # start_date = '2020-01-01'
        # end_date = '2020-12-31'

        if ticker in ticker_dictionary:
            tkr = ticker_dictionary[ticker]
        else: tkr = ticker

        data = yf.download(  # or pdr.get_data_yahoo(...
                # tickers list or string as well
                tickers = tkr,

                # use "period" instead of start/end
                # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                # (optional, default is '1mo')
                period = "1y",

        #         start = start_date,
        #         end = end_date,

                # fetch data by interval (including intraday if period < 60 days)
                # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                # (optional, default is '1d')
                interval = "1d",

                # group by ticker (to access via data['SPY'])
                # (optional, default is 'column')
                group_by = 'ticker',

                # adjust all OHLC automatically
                # (optional, default is False)
                auto_adjust = True,

                # download pre/post regular market hours data
                # (optional, default is False)
                prepost = True,

                # use threads for mass downloading? (True/False/Integer)
                # (optional, default is True)
                threads = True,

                # proxy URL scheme use use when downloading?
                # (optional, default is None)
                proxy = None
            )

        # data.to_csv('BTC_1hr_prices.csv')
        # data.head(3)
    #     print(data.tail(5))

        cur_price_dict[ticker] = data['Close'][-1]

        min_ = []
        max_ = []
        median = []
        top_q = []
        bottom_q = []
        realized = []

        for window in windows:

            # get a dataframe with realized volatility
            estimator = realized_vol(window=window, price_data=data)
    #         print(estimator)

            # append the summary stats to a list
            min_.append(estimator.min())
            max_.append(estimator.max())
            median.append(estimator.median())
            top_q.append(estimator.quantile(quantiles[1]))
            bottom_q.append(estimator.quantile(quantiles[0]))
            realized.append(estimator[-1])
    #         print(min_)

        pct_list = []

        pct30 = (realized[0] - bottom_q[0]) / (top_q[0] - bottom_q[0]) * 100
        pct_list.append(pct30)
        pct60 = (realized[1] - bottom_q[1]) / (top_q[1] - bottom_q[1]) * 100
        pct_list.append(pct60)
        pct90 = (realized[2] - bottom_q[2]) / (top_q[2] - bottom_q[2]) * 100
        pct_list.append(pct90)
        pct120 = (realized[3] - bottom_q[3]) / (top_q[3] - bottom_q[3]) * 100
        pct_list.append(pct120)

    #     print(pct30)
    #     print(pct60)
    #     print(pct90)
    #     print(pct120)
    #     print(pct_list)

        avg_pct = sum(pct_list)/len(pct_list)
    #     print(avg_pct)

        completed_tkr_list.append(ticker)
        realized_vol_list.append(round(realized[1] * 100,2)) # this is realized vol for 60 day window
        avg_pct_list.append(round(avg_pct,2))
        pct30_list.append(round(pct30,2))
        pct60_list.append(round(pct60,2))
        pct90_list.append(round(pct90,2))
        pct120_list.append(round(pct120,2))

        counter += 1
        
    except Exception as e:
        print(f'{tkr} was not found.')
        print(f'Exception: {e}')
        print(f'Error:\n{sys.exc_info()}')
        counter += 1

Processing SPY, 1 of 297...
[*********************100%***********************]  1 of 1 completed
Processing QQQ, 2 of 297...
[*********************100%***********************]  1 of 1 completed
Processing SPXU, 3 of 297...
[*********************100%***********************]  1 of 1 completed
Processing AAPL, 4 of 297...
[*********************100%***********************]  1 of 1 completed
Processing AMZN, 5 of 297...
[*********************100%***********************]  1 of 1 completed
Processing TSLA, 6 of 297...
[*********************100%***********************]  1 of 1 completed
Processing AMD, 7 of 297...
[*********************100%***********************]  1 of 1 completed
Processing VIX, 8 of 297...
[*********************100%***********************]  1 of 1 completed
Processing IWM, 9 of 297...
[*********************100%***********************]  1 of 1 completed
Processing HYG, 10 of 297...
[*********************100%***********************]  1 of 1 completed
Processing META, 11 of 29

[*********************100%***********************]  1 of 1 completed
Processing DS, 85 of 297...
[*********************100%***********************]  1 of 1 completed
Processing CTLP, 86 of 297...
[*********************100%***********************]  1 of 1 completed
Processing VRM, 87 of 297...
[*********************100%***********************]  1 of 1 completed
Processing HGEN, 88 of 297...
[*********************100%***********************]  1 of 1 completed
Processing AGEN, 89 of 297...
[*********************100%***********************]  1 of 1 completed
Processing ACRX, 90 of 297...
[*********************100%***********************]  1 of 1 completed
Processing NMTR, 91 of 297...
[*********************100%***********************]  1 of 1 completed
Processing CRNT, 92 of 297...
[*********************100%***********************]  1 of 1 completed
Processing OEG, 93 of 297...
[*********************100%***********************]  1 of 1 completed
Processing THCA, 94 of 297...
[*************

[*********************100%***********************]  1 of 1 completed
Processing NLOK, 167 of 297...
[*********************100%***********************]  1 of 1 completed
Processing DAL, 168 of 297...
[*********************100%***********************]  1 of 1 completed
Processing GLBS, 169 of 297...
[*********************100%***********************]  1 of 1 completed
Processing CLF, 170 of 297...
[*********************100%***********************]  1 of 1 completed
Processing ADVM, 171 of 297...
[*********************100%***********************]  1 of 1 completed
Processing AEM, 172 of 297...
[*********************100%***********************]  1 of 1 completed
Processing TEF, 173 of 297...
[*********************100%***********************]  1 of 1 completed
Processing IPOD, 174 of 297...
[*********************100%***********************]  1 of 1 completed
Processing GFI, 175 of 297...
[*********************100%***********************]  1 of 1 completed
Processing AQMS, 176 of 297...
[****

[*********************100%***********************]  1 of 1 completed
Processing MRO, 249 of 297...
[*********************100%***********************]  1 of 1 completed
Processing TTOO, 250 of 297...
[*********************100%***********************]  1 of 1 completed
Processing STWD, 251 of 297...
[*********************100%***********************]  1 of 1 completed
Processing CVE, 252 of 297...
[*********************100%***********************]  1 of 1 completed
Processing RKT, 253 of 297...
[*********************100%***********************]  1 of 1 completed
Processing HMLP, 254 of 297...
[*********************100%***********************]  1 of 1 completed
Processing KOS, 255 of 297...
[*********************100%***********************]  1 of 1 completed
Processing RVNC, 256 of 297...
[*********************100%***********************]  1 of 1 completed
Processing CLR, 257 of 297...
[*********************100%***********************]  1 of 1 completed
Processing PLAY, 258 of 297...
[****

In [7]:
import pandas as pd

pct_dictionary = {
    'ticker':completed_tkr_list,
    'realized_vol':realized_vol_list,
    'avg_pct':avg_pct_list,
    'pct30':pct30_list,
    'pct60':pct60_list,
    'pct90':pct90_list,
    'pct120':pct120_list
}

cheap_df = pd.DataFrame(pct_dictionary)
cheap_df = cheap_df.sort_values(['avg_pct'])
cheap_df.to_csv('../0_data_dump/optionistics/realized_vol.csv')
print(f'Size of cheap_df is {len(cheap_df)}')
cheap_df

Size of cheap_df is 297


,ticker,realized_vol,avg_pct,pct30,pct60,pct90,pct120
256,CLR,27.14,-156.23,-112.04,-351.33,-104.61,-56.93
29,UVXY,72.35,-134.56,2.02,-71.14,-378.54,-90.59
46,IPOF,1.90,-134.38,-63.46,-81.08,-184.39,-208.61
5,TSLA,48.25,-113.94,-112.26,-147.11,-172.31,-24.07
54,BBD,30.17,-104.87,-47.58,-56.63,-93.80,-221.48
...,...,...,...,...,...,...,...
161,HUT,121.51,299.41,173.44,249.56,340.11,434.54
102,TEVA,74.93,374.08,25.12,558.75,510.50,401.94
242,VTNR,164.93,375.89,486.40,365.78,289.28,362.09
274,FAZE,203.54,518.92,450.58,533.69,535.14,556.26


In [8]:
import glob
import os

# Get the latest file from the directory
list_of_files = glob.glob('../0_inputs/asset_screeners/Finviz_Data/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

../0_inputs/asset_screeners/Finviz_Data\finviz (64).csv


In [9]:
import pandas as pd
import numpy as np

col_names = {
    'Ticker':'ticker',
    'Float Short':'float_short'
}

# Bring finviz.csv file into dataframe
finviz_df = pd.read_csv(latest_file)
finviz_df = finviz_df.rename(columns=col_names)
finviz_df = finviz_df[['ticker','float_short']]
finviz_df['float_short'] = finviz_df['float_short'].str.slice(0,-1) # remove the % character from the end of the string
finviz_df['float_short'] = finviz_df['float_short'].astype(str).astype(float) # convert to float for proper sorting
finviz_df = finviz_df.sort_values(['float_short'],ascending=False)

# Add Relative Volume Logic here
finviz_df['float_short_pt'] = 0
finviz_df['float_short_pt'] = np.select(condlist=[(finviz_df['float_short']>finviz_df['float_short'].quantile(0.85))],
                           choicelist=[1],
                           default=0)

# finviz_df = finviz_df[finviz_df['float_short'] > finviz_df['float_short'].quantile(.85)]
# finviz_df2 = finviz_df[finviz_df['float_short_pt']==1]
print(f'Size of finviz_df is {len(finviz_df)}.')
finviz_df.head(3)

Size of finviz_df is 8531.


,ticker,float_short,float_short_pt
2530,ESTE,51.14,1
3822,ICPT,46.84,1
2684,FAZE,39.95,1


In [10]:
# Optional for if internet is running slow
# cheap_df = pd.read_csv('../0_data_dump/optionistics/realized_vol_output.csv')
# cheap_df = cheap_df.drop('Unnamed: 0', axis=1)

# Add Relative Volume Logic here
cheap_df['cheap_pt'] = 0
cheap_df['cheap_pt'] = np.select(condlist=[(cheap_df['avg_pct'] < 0)],
                           choicelist=[1],
                           default=0)

print(f'Size of cheap_df is {len(cheap_df)}')
cheap_df.head(3)

Size of cheap_df is 297


,ticker,realized_vol,avg_pct,pct30,pct60,pct90,pct120,cheap_pt
256,CLR,27.14,-156.23,-112.04,-351.33,-104.61,-56.93,1
29,UVXY,72.35,-134.56,2.02,-71.14,-378.54,-90.59,1
46,IPOF,1.90,-134.38,-63.46,-81.08,-184.39,-208.61,1


Implied volatility premium or discount calculations here.

In [11]:
import yfinance as yf
import datetime
import sys

today_date = datetime.datetime.now()
target_date = today_date + datetime.timedelta(days=75) # calendar days

# ticker_list = ['AAPL','TSLA','MSFT'] # test list

ticker_list2 = []
expiry_list = []
strike_list = []
iv_list = []
counter = 1

for ticker in ticker_list:
    
    try:
        print(f'\nProcessing {ticker}, {counter} of {len(ticker_list)}...')
        
        # options_data = yf.Ticker("MSFT") # test input
        options_data = yf.Ticker(ticker)
        option_dates_list = options_data.options

        closest_date = today_date

        for x in option_dates_list:
            x = datetime.datetime.strptime(x, '%Y-%m-%d')
        #     print(x,',',(x-target_date).days)
            if (abs((x - target_date).days)) < abs(((closest_date - target_date).days)):
                closest_date = x

        # print(closest_date)
        # days_to_expiration = (closest_date - today_date).days
        # print(days_to_expiration)
        closest_date = closest_date.strftime("%Y-%m-%d")
        print(f'Closest expiry is: {closest_date}')

        opt = options_data.option_chain(closest_date)
        strike_price_np_array = opt[0]['strike'].to_numpy()

        if ticker in cur_price_dict:
            cur_price = cur_price_dict[ticker]
        else:
            data = yf.download(
                tickers = tkr,
                period = "5d",
                interval = "1d",
                group_by = 'ticker',
                auto_adjust = True,
                prepost = True,
                threads = True,
                proxy = None
            )
            cur_price = data['Close'][-1]
        
#         cur_price = 280 # test input
        closest_strike = 0

        strike_ctr = 0
        
        for x in strike_price_np_array:
            if abs(cur_price - x) < abs(cur_price - closest_strike):
                closest_strike = x
                strike_index = strike_ctr
            strike_ctr += 1

        print(f'Closest strike price is: {closest_strike}')

        strike_df = opt[0][opt[0]['strike']==closest_strike]
        implied_vol = round(strike_df['impliedVolatility'][strike_index] * 100,2)
        print(f'Implied volatility is: {implied_vol}')

        ticker_list2.append(ticker)
        expiry_list.append(closest_date)
        strike_list.append(closest_strike)
        iv_list.append(implied_vol)
        counter += 1
    
    except Exception as e:
        print(f'{ticker}, {counter} not found.')
        print(f'Exception is {e}')
        print(f'Error is: {sys.exc_info()}')
        counter += 1


Processing SPY, 1 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 400.0
Implied volatility is: 23.12

Processing QQQ, 2 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 301.0
Implied volatility is: 30.34

Processing SPXU, 3 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 16.0
Implied volatility is: 69.09

Processing AAPL, 4 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 155.0
Implied volatility is: 32.96

Processing AMZN, 5 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 130.0
Implied volatility is: 43.98

Processing TSLA, 6 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 286.67
Implied volatility is: 58.05

Processing AMD, 7 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 85.0
Implied volatility is: 53.37

Processing VIX, 8 of 297...
Closest expiry is: 2022-09-08
VIX, 8 not found.
Exception is Expiration `2022-09-08` cannot be found. Available expiration are: []
Erro

Closest expiry is: 2022-12-16
Closest strike price is: 0.5
Implied volatility is: 198.44

Processing BIOR, 68 of 297...
Closest expiry is: 2022-10-21
Closest strike price is: 0.5
Implied volatility is: 221.88

Processing RIG, 69 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 3.5
Implied volatility is: 83.59

Processing PFE, 70 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 47.5
Implied volatility is: 29.64

Processing MMAT, 71 of 297...
Closest expiry is: 2022-10-21
Closest strike price is: 1.0
Implied volatility is: 125.0

Processing C, 72 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 50.0
Implied volatility is: 32.57

Processing MULN, 73 of 297...
Closest expiry is: 2022-10-21
Closest strike price is: 0.5
Implied volatility is: 150.0

Processing CGC, 74 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 3.5
Implied volatility is: 108.98

Processing AGNC, 75 of 297...
Closest expiry is: 2022-12-16
Closest strike pric

Closest expiry is: 2022-12-16
Closest strike price is: 7.5
Implied volatility is: 31.25

Processing FENG, 128 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 5.0
Implied volatility is: 9.18

Processing AGCB, 129 of 297...
Closest expiry is: 2022-10-21
Closest strike price is: 10.0
Implied volatility is: 0.78

Processing WISH, 130 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 1.0
Implied volatility is: 114.84

Processing GM, 131 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 40.0
Implied volatility is: 47.29

Processing ABEO, 132 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 5.0
Implied volatility is: 12.5

Processing ASHR, 133 of 297...
Closest expiry is: 2022-10-21
Closest strike price is: 30.0
Implied volatility is: 21.92

Processing PARA, 134 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 22.5
Implied volatility is: 49.71

Processing HTZ, 135 of 297...
Closest expiry is: 2022-12-16
Closest str

Closest strike price is: 10.0
Implied volatility is: 43.95

Processing EVOK, 190 of 297...
Closest expiry is: 2022-10-21
Closest strike price is: 2.5
Implied volatility is: 44.53

Processing FSK, 191 of 297...
Closest expiry is: 2022-10-21
Closest strike price is: 20.0
Implied volatility is: 17.97

Processing BODY, 192 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 2.5
Implied volatility is: 89.06

Processing UNIT, 193 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 9.0
Implied volatility is: 43.16

Processing SKLZ, 194 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 1.5
Implied volatility is: 127.34

Processing BB, 195 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 6.0
Implied volatility is: 68.16

Processing CIM, 196 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 8.0
Implied volatility is: 34.47

Processing M, 197 of 297...
Closest expiry is: 2022-11-18
Closest strike price is: 18.0
Implied volati

Closest strike price is: 0
TTOO, 250 not found.
Exception is 15
Error is: (<class 'KeyError'>, KeyError(15), <traceback object at 0x00000185C72690C8>)

Processing STWD, 251 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 23.0
Implied volatility is: 22.05

Processing CVE, 252 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 18.0
Implied volatility is: 54.93

Processing RKT, 253 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 8.0
Implied volatility is: 59.86

Processing HMLP, 254 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 10.0
Implied volatility is: 18.26

Processing KOS, 255 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 5.0
Implied volatility is: 86.72

Processing RVNC, 256 of 297...
Closest expiry is: 2022-10-21
Closest strike price is: 25.0
Implied volatility is: 69.78

Processing CLR, 257 of 297...
Closest expiry is: 2022-12-16
Closest strike price is: 70.0
Implied volatility is: 37.93

Process

In [12]:
print('Completed.')

Completed.


Create a dataframe for the implied volatility data.

In [13]:
iv_dictionary = {
    'ticker':ticker_list2,
    'expiry':expiry_list,
    'strike':strike_list,
    'implied_vol':iv_list
}

iv_df = pd.DataFrame(iv_dictionary)
iv_df["implied_vol"] = iv_df["implied_vol"].astype(str).astype(float)
iv_df = iv_df.sort_values(['implied_vol'],ascending=False)
iv_df.to_csv('../0_data_dump/optionistics/implied_vols.csv')
print(f'Size of iv_df is {len(iv_df)}.')
iv_df.head(3)

Size of iv_df is 268.


,ticker,expiry,strike,implied_vol
83,AGEN,2022-11-18,2.5,695.31
128,AMRN,2022-11-18,1.0,275.00
110,CLVS,2022-10-21,1.0,226.56


In [14]:
print('Completed.')

Completed.


In [15]:
# for col in final_df.columns:
#     print(col)

In [16]:
final_df = pd.merge(cheap_df,finviz_df,on='ticker',how='left')
print(len(final_df))
final_df = pd.merge(final_df,optionistics_data,on='ticker',how='outer')
print(len(final_df))
final_df = pd.merge(final_df,iv_df,on='ticker',how='outer')
print(len(final_df))

final_df['iv_premium'] = final_df['implied_vol'] - final_df['realized_vol']

final_df['iv_prem_pt'] = 0
final_df['iv_prem_pt'] = np.select(condlist=[(final_df['iv_premium']>final_df['iv_premium'].quantile(0.85))],
                           choicelist=[1],
                           default=0)

final_df['oi_pct_chg_pts'] = final_df['oi_pct_chg_pts'].fillna(0)
final_df['cheap_pt'] = final_df['cheap_pt'].fillna(0)
final_df['total_pts'] = final_df['cheap_pt'] + final_df['float_short_pt'] + final_df['oi_pct_chg_pts'] + final_df['iv_prem_pt']
final_df = final_df[['ticker','expiry','strike','total_pts','iv_prem_pt','cheap_pt','oi_pct_chg_pts','float_short_pt','iv_premium','avg_pct','float_short','oi_pct_chg','p_c_ratio']]
final_df = final_df.sort_values(by = ['total_pts','float_short'],ascending=[False,False])
final_df = final_df[final_df['total_pts'] > 0]
final_df.to_csv('../0_data_dump/asset_screener/options_output.csv')
final_df.head(20)

297
297
297


,ticker,expiry,strike,total_pts,iv_prem_pt,cheap_pt,oi_pct_chg_pts,float_short_pt,iv_premium,avg_pct,float_short,oi_pct_chg,p_c_ratio
57,NKLA,2022-11-18,5.00,4.0,1,1,1.0,1.0,24.63,-13.31,30.07,8.58,0.92
56,GME,2022-11-18,26.25,4.0,1,1,1.0,1.0,25.01,-13.51,20.72,7.06,1.27
23,WISH,2022-12-16,1.00,3.0,1,1,0.0,1.0,43.14,-45.88,10.66,NaN,NaN
256,ALT,2022-11-18,19.00,3.0,1,0,1.0,1.0,53.71,114.73,10.40,30.49,0.44
20,VSTO,2022-11-18,27.50,3.0,0,1,1.0,1.0,21.22,-49.17,9.30,7.62,2.10
71,CHRW,2022-11-18,115.00,3.0,0,1,1.0,1.0,5.63,-1.84,8.40,9.27,0.60
295,FAZE,2022-10-21,12.50,2.0,0,0,1.0,1.0,-129.71,518.92,39.95,8.49,0.79
175,SPWR,2022-12-16,28.00,2.0,0,0,1.0,1.0,-2.28,55.70,38.54,11.08,0.65
10,SIRI,2022-11-18,6.00,2.0,0,1,0.0,1.0,9.00,-89.37,29.64,NaN,NaN
125,VUZI,2022-10-21,8.00,2.0,0,0,1.0,1.0,-10.19,32.36,26.99,8.22,0.60


In [17]:
toc = time.perf_counter()
total_time = toc - tic

print(f'Program completed in {round(total_time,2)} seconds or {round(total_time/60,2)} minutes on {datetime.datetime.now()}.')

Program completed in 219.41 seconds or 3.66 minutes on 2022-09-08 22:36:28.123691.
